In [269]:
import ast
from functools import partial
import networkx as nx
import numpy as np
from typing import Dict
import matplotlib.pyplot as plt
from dataclasses import dataclass
from mpi4py import MPI
from repast4py import context as ctx
from repast4py import core, random, schedule, logging, parameters
from repast4py.network import write_network, read_network

In [270]:
import json
import zlib
import base64
from collections import defaultdict
model = None


class RumorAgent(core.Agent):

    def __init__(self, nid: int, agent_type: int, rank: int, received_rumor=False, shadow=None):
        super().__init__(nid, agent_type, rank)
        self.received_rumor = received_rumor
        self.shadow = shadow or {}

    
def create_rumor_agent(nid, agent_type, rank, **kwargs):
    shadow_data = {}
    if 'data' in kwargs:  # New compressed format
        encoded_str = kwargs['data']
        compressed = base64.b64decode(encoded_str)
        json_str = zlib.decompress(compressed).decode('utf-8')
        raw_shadow = json.loads(json_str)['shadow']
        # Convert string keys to tuples and integer layer IDs
        shadow_data = {
            int(layer): {
                ast.literal_eval(conn_key): weight 
                for conn_key, weight in conns.items()
            }
            for layer, conns in raw_shadow.items()
        }
    return RumorAgent(nid, agent_type, rank, received_rumor=None, shadow=shadow_data)

def restore_agent(agent_data):
    uid = agent_data[0]
    received_rumor = agent_data[1]
    shadow = agent_data[2] if len(agent_data) > 2 else {}  # Handle cases where shadow might not be saved
    return RumorAgent(uid[0], uid[1], uid[2], received_rumor, shadow)

In [271]:
class Model:
    def __init__(self, comm, params):
        self.runner = schedule.init_schedule_runner(comm)
        self.runner.schedule_stop(params['stop.at'])

        fpath = params['network_file']
        self.context = ctx.SharedContext(comm)
        read_network(fpath, self.context, create_rumor_agent, restore_agent)
        self.net = self.context.get_projection('rumor_network')
        for agent in self.context.agents():
            print(agent.uid, agent.shadow)
    def start(self):
        self.runner.execute()

In [ ]:
def run(params: Dict):
    global model
    model = Model(MPI.COMM_WORLD, params)
    model.start()


if __name__ == "__main__":
    params = {
        'network_file': 'layer1.txt',
        'initial_rumor_count': 5,
        'stop.at': 5,
        'rumor_probability': 0.1,
        'counts_file': 'output/rumor_counts.csv'
    }
    run(params)

(0, 0, 2) {0: {}, 1: {(2, 0, 1): 1.0}}
(1, 0, 1) {0: {(11, 0, 2): 1.0}, 1: {}}
(2, 0, 1) {0: {(13, 0, 2): 1.0}, 1: {(0, 0, 2): 1.0}}
(3, 0, 1) {0: {}, 1: {}}
(4, 0, 2) {0: {}, 1: {}}
(5, 0, 1) {0: {(12, 0, 2): 1.0}, 1: {}}
(6, 0, 1) {0: {(16, 0, 0): 1.0}, 1: {(12, 0, 2): 1.0, (15, 0, 1): 1.0}}
(7, 0, 0) {0: {(11, 0, 2): 1.0}, 1: {(17, 0, 0): 1.0}}
(8, 0, 0) {0: {}, 1: {}}
(9, 0, 1) {0: {(13, 0, 2): 1.0}, 1: {}}
(10, 0, 0) {0: {(13, 0, 2): 1.0}, 1: {(19, 0, 0): 1.0}}
(11, 0, 2) {0: {(1, 0, 1): 1.0, (7, 0, 0): 1.0, (14, 0, 0): 1.0, (18, 0, 2): 1.0}, 1: {}}
(12, 0, 2) {0: {(5, 0, 1): 1.0}, 1: {(6, 0, 1): 1.0}}
(13, 0, 2) {0: {(2, 0, 1): 1.0, (9, 0, 1): 1.0, (10, 0, 0): 1.0}, 1: {(15, 0, 1): 1.0}}
(14, 0, 0) {0: {(11, 0, 2): 1.0}, 1: {}}
(15, 0, 1) {0: {}, 1: {(6, 0, 1): 1.0, (13, 0, 2): 1.0}}
(16, 0, 0) {0: {(6, 0, 1): 1.0}, 1: {}}
(17, 0, 0) {0: {}, 1: {(7, 0, 0): 1.0}}
(18, 0, 2) {0: {(11, 0, 2): 1.0}, 1: {}}
(19, 0, 0) {0: {}, 1: {(10, 0, 0): 1.0}}
